# Hypothesis Testing on IFSC and 8a.nu Climbing Data

## Introduction

This  explores statistical relationships between outdoor and competition climbing performance using a dataset derived from two key sources: **IFSC** (competition climbing data) and **8a.nu** (outdoor climbing data), which was scraped and merged in the notebooks located in the `01_data_collection` folder. For details on the data collection and merging process, refer to those notebooks. By analyzing this unified dataset, we aim to investigate whether measurable trends or correlations exist between climbers’ achievements across different contexts.

## Objectives

The main goals of this analysis are to test the following hypotheses:

- **Outdoor vs. Competition Performance**
   Do climbers with stronger outdoor climbing metrics—such as higher maximum grade or having a 8c+ ascent—also perform better in competitions?
   *Separate hypothesis tests will be conducted for each competition discipline: bouldering, lead, and combined.*

- **Interdisciplinary Competition Correlation**
   Is there a statistically significant correlation between performance in different IFSC competition disciplines (e.g., do high lead scores correlate with high boulder scores)?

- **Outdoor Performance Correlation**
   Within outdoor climbing, is there a meaningful correlation between the metrics (e.g., do higher average grades correlate with higher best grade climbed)?


In [1]:
import pandas as pd
import scipy.stats as stats

In [2]:
df = pd.read_csv('../data/final_data.csv')

## Hypothesis Test 1: Outdoor Grade 8c+ vs. Competition Performance

Do climbers perform better in **competitions** if they have climbed an **outdoor grade of 8c+ or above**?

#### Hypotheses:

- **$H_0$**: Climbing an outdoor grade of 8c+ or above has no effect on competition performance in a specific discipline.
  μ₁ = μ₂
  *(Where μ₁ = mean competition points in specific discipline for climbers who have climbed 8c+ or above, μ₂ = mean boulder points for those who have not.)*

- **$H_a$**:  Climbers who have climbed an outdoor grade of 8c+ or above perform better on competitions in said specific discipline.
  μ₁ > μ₂

#### Methodology
- **Test Type:** One-tailed, **Two-sample T-Test**
$$
t = \frac{\bar{X}_A - \bar{X}_B}{\sqrt{\frac{s_A^2}{n_A} + \frac{s_B^2}{n_B}}}
$$
- **Grouping Variable:** Whether the climber has climbed 8c+ or above
- **Dependent Variable:** Competition points (tested separately for 3 disciplines)

This test aims to determine whether there is a statistically significant difference in competition performance between climbers who have reached the 8c+ outdoor benchmark and those who have not.
To conduct this analysis, a Python function is defined and called separately for each discipline to evaluate the hypothesis:

In [3]:
def perform_t_test(df, points_column, alpha=0.05):
    # Filter athletes with non-zero points for the discipline
    athletes = df[df[points_column] > 0]

    # Sample 1: Climbers with 8c+ or above
    sample_1 = athletes[athletes['count_8c_plus'] > 0][points_column]
    # Sample 2: Climbers without 8c+ (count_8c_plus is NaN)
    sample_2 = athletes[athletes['count_8c_plus'].isna()][points_column]

    # Perform one-tailed t-test (Hₐ: μ₁ > μ₂)
    t_stat, p_value = stats.ttest_ind(sample_1, sample_2, equal_var=False, alternative='greater')

    # Print results
    discipline = points_column.replace('_points', '').capitalize()
    print(f"\n{discipline} Discipline:")
    print(f"t-statistic: {t_stat:.4f}")
    print(f"One-tailed p-value: {p_value:.4f}")
    print(f"Sample sizes: With 8c+ = {len(sample_1)}, Without 8c+ = {len(sample_2)}")

    # Interpret results
    if p_value < alpha:
        print(f"Reject H₀: Climbers with 8c+ perform significantly better in {discipline.lower()} discipline.")
    else:
        print(f"Fail to reject H₀: No significant evidence that climbers with 8c+ perform better in {discipline.lower()} discipline.")

In [4]:
# List of disciplines (points columns)
disciplines = ['boulder_points', 'lead_points', 'combined_points']

# Perform t-test for each discipline
for points_column in disciplines:
    perform_t_test(df, points_column)


Boulder Discipline:
t-statistic: 1.8086
One-tailed p-value: 0.0382
Sample sizes: With 8c+ = 41, Without 8c+ = 388
Reject H₀: Climbers with 8c+ perform significantly better in boulder discipline.

Lead Discipline:
t-statistic: 2.3655
One-tailed p-value: 0.0105
Sample sizes: With 8c+ = 51, Without 8c+ = 265
Reject H₀: Climbers with 8c+ perform significantly better in lead discipline.

Combined Discipline:
t-statistic: 2.4045
One-tailed p-value: 0.0103
Sample sizes: With 8c+ = 32, Without 8c+ = 163
Reject H₀: Climbers with 8c+ perform significantly better in combined discipline.


#### Conclusion:
Overall, having climbed a grade of 8c+ or above appears to be a strong indicator of success in competition climbing. While all three disciplines show statistically significant improvements, the effect is especially pronounced in the **lead** and **combined** categories, suggesting that high-level outdoor climbing experience may translate more directly to these formats than to **bouldering**.


## Hypothesis Test 2: Highest Grade Achieved in Outdoor Climbing vs. Competition Performance

Is there a statistically significant correlation between the **highest outdoor grade** a climber has achieved and their **performance in competition climbing**?

#### Hypotheses:

- **$H_0$**: There is **no correlation** between the highest grade ever climbed in outdoor climbing and competition performance.  ρ = 0
  *(Where ρ is the population correlation coefficient between `highest_grade` and a given competition performance metric, such as `boulder_points`.)*

- **$H_a$**: There **is a correlation** between the highest grade ever climbed in outdoor climbing and competition performance.  ρ ≠ 0

#### Methodology

- **Test Type: Pearson correlation test**
- **Independent Variable:** `highest_grade` (converted to numeric scale while scraping)
- **Dependent Variables:** Competition points (`boulder_points`, `lead_points`, and `combined_points`)

To conduct this analysis, a Python function is defined and called separately for each discipline:


In [5]:
def test_grade_correlation(df, points_column, alpha=0.05):
    # Filter athletes who competed in the discipline and have a recorded highest_grade
    athletes = df[(df[points_column] > 0) & (df['highest_grade'] > 0)]

    # Perform Pearson correlation test
    r, p_value = stats.pearsonr(athletes[points_column], athletes['highest_grade'])

    # Print results
    discipline = points_column.replace('_points', '').capitalize()
    print(f"\n{discipline} Discipline:")
    print(f"Pearson Correlation Coefficient: {r:.4f}")
    print(f"Two-tailed p-value: {p_value:.4f}")
    print(f"Sample size: {len(athletes)}")

    # Interpret results
    if p_value < alpha:
        print(f"Reject H₀: Significant correlation between highest grade and {discipline.lower()} performance.")
    else:
        print(f"Fail to reject H₀: No significant correlation between highest grade and {discipline.lower()} performance.")

In [6]:
# List of disciplines (points columns)
disciplines = ['boulder_points', 'lead_points', 'combined_points']

# Perform correlation test for each discipline
for points_column in disciplines:
    test_grade_correlation(df, points_column)


Boulder Discipline:
Pearson Correlation Coefficient: 0.2791
Two-tailed p-value: 0.0024
Sample size: 116
Reject H₀: Significant correlation between highest grade and boulder performance.

Lead Discipline:
Pearson Correlation Coefficient: 0.1689
Two-tailed p-value: 0.0699
Sample size: 116
Fail to reject H₀: No significant correlation between highest grade and lead performance.

Combined Discipline:
Pearson Correlation Coefficient: 0.2915
Two-tailed p-value: 0.0106
Sample size: 76
Reject H₀: Significant correlation between highest grade and combined performance.


#### Conclusion: Correlation Between Highest Outdoor Grade and Competition Performance

The results of the Pearson correlation tests show that there **is a statistically significant correlation** between the **highest outdoor grade** a climber has achieved and their **competition performance** in two out of the three disciplines:

- In the **bouldering and combined** disciplines, a **moderate positive correlation** was found, indicating that climbers who have sent harder outdoor routes tend to perform better in bouldering and combined competitions.
- However, for the **lead** discipline, the correlation (r = 0.1689) was **not statistically significant** at the 0.05 level (p = 0.0699), meaning we cannot confidently conclude a relationship in this case, which is surprising since outdoor sport climbing is most similar to lead climbing.

These results suggest that **outdoor climbing experience—particularly reaching higher grades—may be linked to improved competition performance**, especially in **bouldering** and **combined** formats. However, the absence of significance in lead climbing implies that the relationship may be more complex or non-linear.


## Hypothesis Test 3: Performance Across IFSC Competition Disciplines

Is there a statistically significant correlation between performance in different **IFSC competition disciplines**?

#### Hypotheses

- **$H_0$**: There is **no correlation** between performance in different competition disciplines.   Mathematically:  ρ = 0
  *(Where ρ is the population correlation coefficient eg. between `boulder_points` and `lead_points`)*

- **$H_a$**:  There **is a correlation** between performance in different competition disciplines. Mathematically:  ρ ≠ 0

#### Methodology

- **Test Type:** Pearson correlation test

This test aims to assess whether success in one competition discipline is predictive of success in another, which could indicate shared skillsets or training transfer across disciplines.
To conduct this analysis, a Python function is defined and called separately for each discipline to evaluate the hypothesis:

In [7]:
# Filter athletes who competed in both bouldering and lead climbing
athletes_both = df[(df['boulder_points'] > 0) & (df['lead_points'] > 0)]

# Compute Pearson correlation
r, p_value = stats.pearsonr(athletes_both['boulder_points'], athletes_both['lead_points'])

print(f"Pearson Correlation Coefficient: {r:.4f}")
print(f"p-value: {p_value:.4f}")

Pearson Correlation Coefficient: 0.5940
p-value: 0.0000
